# Pandas: Tabular Data in Python

## Objectives

* Create `Series` and `DataFrame`s from Python data types. 
* Create `DataFrame`s from on disk data.
* Index and Slice `pandas` objects.
* Aggregate data in `DataFrame`s using `groupby`.

## What is Pandas?

A Python library providing data structures and data analysis tools for tabular data of many types.

## Benefits

  * Efficient storage and processing of data.
  * Includes many built in functions for data transformation, aggregations, and plotting.
  * Great for exploratory work.

## Not so greats

  * Does not scale terribly well to large datasets.

## Documentation:

The documentation for pandas is here:

  * http://pandas.pydata.org/pandas-docs/stable/index.html
  
Particularly important reads (eventaully) are:

  * [Indexing and Selecting](https://pandas.pydata.org/pandas-docs/stable/indexing.html)
  * [Advanced Indexing](http://pandas.pydata.org/pandas-docs/version/0.20.3/advanced.html#advanced-mi-slicers)
  * [Group-by](https://pandas.pydata.org/pandas-docs/stable/groupby.html)

## Standard Imports

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

## Numpy: A Quick Primer

`pandas` is built out of data types from `numpy` a lower level library we will be studying soon.

For now, it is sufficient to know that `numpy`s main feature is a very efficient data structure, the `array`.

In [2]:
x = np.array([0, 1, 2, 3, 4, 5])
x

array([0, 1, 2, 3, 4, 5])

Arrays can be processed very efficiently.

In [3]:
x.sum()  # <-- As efficient as possible way to sum these numbers in python.

15

Arrays can be multi-dimensional.  A **two-dimensional array** is called a **matrix**.

In [4]:
M = np.array([
    [0, 1, 2],
    [1, 2, 3],
    [2, 3, 4]
])

M

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [5]:
print(x.shape)
print(M.shape)

(6,)
(3, 3)


### Drawbacks of Numpy as a General Data Analysis Tool

#### Numpy arrays can only store homogeneous data

In [6]:
x = np.array([
    [2.0, 3.4, "Jack"],
    [1.0, 0.4, "Matt"],
    [5.0, 9.4, "Miles"]
])

That seemed to work...

In [7]:
x.dtype

dtype('<U32')

What?

Numpy has chosen to store our array as **uncode strings**.  **Even the numbers are now strings!**

In [8]:
x[0, 0]  # <- It is a string! It is a string!

'2.0'

Arithmetic operations that should work do not.  Here is an attempt at a column sum!

In [9]:
# Column sum
x.sum(axis=1)

TypeError: cannot perform reduce with flexible type

This happens because numpy arrays are **homogeneous**.  All the data in an array (even in different columns) must be of the same datatype!

#### Numpy Arrays only Accept Integer Indexes

You cannot assign column or row names to numpy arrays.  This can make programming for data analysis a nightmare.

## Getting Data into Pandas

### Creating DataFrames from Python Objects

You can think of DataFrames as labeled (columns) and indexed (rows) matrices. 

We can create DataFrames from numpy arrays and list of lists with provided labels and indices.

In [10]:
pd.DataFrame(
    [[1, 2, 3], [4, 5, 6]], 
    columns=['a', 'b', 'c'], 
    index=['foo', 'bar'])

,a,b,c
foo,1,2,3
bar,4,5,6


Alternatively, you can think of DataFrames as a combination of column vectors, so we can create DataFrames from a dictionary of column vectors.  The keys are the column labels, and the values are the vectors.

In [11]:
frame_dict = {'coulmn_1': [1, 2, 3], 'column_2': [10, 11, 12]}
pd.DataFrame(frame_dict, index=['3', '2', '1'])

,coulmn_1,column_2
3,1,10
2,2,11
1,3,12


### Series

If DataFrames are labeled and indexed matrices, then Series are labeled and indexed vectors.

In [13]:
pd.Series([1, 2, 3], index=['a', 'b', 'c'], name='Numbers')

a    1
b    2
c    3
Name: Numbers, dtype: int64

If you create a Series using a dictionary, the keys are treated as indices instead.

In [14]:
pd.Series({'Star': 'Wars', 'Is': 'Boring', 'Please': 'Stop'})

Star        Wars
Is        Boring
Please      Stop
dtype: object

You can take out a Series from a DataFrame.

In [15]:
df = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6]], 
    columns=['a', 'b', 'c'], 
    index=['foo', 'bar'])

print("Data Frame")
print(df)
print()
print("Column 'a'")
print(df['a'])

Data Frame
     a  b  c
foo  1  2  3
bar  4  5  6

Column 'a'
foo    1
bar    4
Name: a, dtype: int64


In [16]:
type(df['c'])

pandas.core.series.Series

... or put a Series into a DataFrame as long as you have matching index.

In [17]:
df['d'] = pd.Series([4, 5], index=['foo', 'bar'])
df

,a,b,c,d
foo,1,2,3,4
bar,4,5,6,5


The elements in the assignment above are matched **by index**, which is a common pattern in Pandas.

In [18]:
# Index flipped from previous example.
#                           v
df['d'] = pd.Series([4, 5], index=['bar', 'foo'])
df

,a,b,c,d
foo,1,2,3,5
bar,4,5,6,4


If no indicies match, missing values are filled into the unmatched spaces.

In [19]:
df['d'] = pd.Series([4, 5], index=['bar', 'baz'])
df

,a,b,c,d
foo,1,2,3,NaN
bar,4,5,6,4.0


We can also put a list/vector into a DataFrame, and here there is no index, so the column is inserted in order.

In [21]:
df['e'] = [1, 2]
df

,a,b,c,d,e
foo,1,2,3,NaN,1
bar,4,5,6,4.0,2


### Load data from csv

A csv (comma separated values) is a file format used to store data separated by a **delimiter**.

A delimiter is a **single character** that delimits boundaries between data elements in a file.  A comma is a traditional choice of delimiter, but a relatively poor one.  Better choices are pipe: `|`, or tab `\t`.

In [22]:
# Pipe separated file.
!head 'playgolf.csv'

Date|Outlook|Temperature|Humidity|Windy|Result
07-01-2014|sunny|85|85|false|Don't Play
07-02-2014|sunny|80|90|true|Don't Play
07-03-2014|overcast|83|78|false|Play
07-04-2014|rain|70|96|false|Play
07-05-2014|rain|68|80|false|Play
07-06-2014|rain|65|70|true|Don't Play
07-07-2014|overcast|64|65|true|Play
07-08-2014|sunny|72|95|false|Don't Play
07-09-2014|sunny|69|70|false|Play


In a bizarre twist of history, comma separated files are often separated by different characters than commas.  There is no consistent convention of using a different file extension, but some people use `.psv` or `.tsv`.

Pandas has a `read_csv` function that loads a delimited file into a `DataFrame`.  The resulting object **must fit in memory**.

In [23]:
golf_df = pd.read_csv('playgolf.csv', delimiter='|')

`DataFrame.head` can be used to view a portion of our new dataframe.

In [24]:
golf_df.head()

,Date,Outlook,Temperature,Humidity,Windy,Result
0,07-01-2014,sunny,85,85,False,Don't Play
1,07-02-2014,sunny,80,90,True,Don't Play
2,07-03-2014,overcast,83,78,False,Play
3,07-04-2014,rain,70,96,False,Play
4,07-05-2014,rain,68,80,False,Play


### Various Summaries

The info class method is useful for checking column types and quickly seeing if you have `NaN`s in the data.

In [25]:
golf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
Date           14 non-null object
Outlook        14 non-null object
Temperature    14 non-null int64
Humidity       14 non-null int64
Windy          14 non-null bool
Result         14 non-null object
dtypes: bool(1), int64(2), object(3)
memory usage: 654.0+ bytes


The describe method will give you a quick sense of the quartiles and distribution.


In [26]:
golf_df.describe()

,Temperature,Humidity
count,14.000000,14.000000
mean,73.571429,80.285714
std,6.571667,9.840486
min,64.000000,65.000000
25%,69.250000,71.250000
50%,72.000000,80.000000
75%,78.750000,88.750000
max,85.000000,96.000000


### Frequency Tables

The `crosstab` function will allow us to quickly take a look at the frequency count between two columns.

In [27]:
pd.crosstab(golf_df['Outlook'], golf_df['Result'])

Result,Don't Play,Play
Outlook,,
overcast,0,4
rain,2,3
sunny,3,2


## Extracting information from DataFrames

### Basic Row and Column Indexing

As we have seen, individual columns may be extracted from a `DataFrame` using the usual `__getitem__` style indexing using the name of the column.  

This is similar to how we index a dictionary.

In [28]:
golf_df['Temperature']

0     85
1     80
2     83
3     70
4     68
5     65
6     64
7     72
8     69
9     75
10    75
11    72
12    81
13    71
Name: Temperature, dtype: int64

We can extract individual values by taking the series out of the matrix, then treating it like a list.

In [29]:
golf_df['Temperature'][0]

85

If you try to index Pandas like a list, with an integer or a slice, it will only operate on the rows.

In [30]:
short_df = golf_df[0:5]
short_df

,Date,Outlook,Temperature,Humidity,Windy,Result
0,07-01-2014,sunny,85,85,False,Don't Play
1,07-02-2014,sunny,80,90,True,Don't Play
2,07-03-2014,overcast,83,78,False,Play
3,07-04-2014,rain,70,96,False,Play
4,07-05-2014,rain,68,80,False,Play


### Boolean / Logical Indexing

Interestingly we can try to index into a dataframe using a list of **booleans** (i.e. `True` and `False` values).

In [31]:
list_of_bools = [True, False, True, False, True]
short_df[list_of_bools]

,Date,Outlook,Temperature,Humidity,Windy,Result
0,07-01-2014,sunny,85,85,False,Don't Play
2,07-03-2014,overcast,83,78,False,Play
4,07-05-2014,rain,68,80,False,Play


We can also **create** a Boolean Series/List by using comparisons on a Series

In [33]:
# A series of booleans.
is_temp_gt_seventy = golf_df['Temperature'] > 70

And them use the result to grab rows of the dataframe.

In [34]:
golf_df[(golf_df['Temperature'] > 70) & (golf_df['Humidity'] < 80)]

,Date,Outlook,Temperature,Humidity,Windy,Result
2,07-03-2014,overcast,83,78,False,Play
10,07-11-2014,sunny,75,70,True,Play
12,07-13-2014,overcast,81,75,False,Play


This is essentially applying a logical condition to select rows from a `DataFrame`.  This is one of the most common patterns in Pandas.

### Double Indexing

Suppose we want to set the value of the `Windy` column where `Temperature > 70` to true (I'm not sure why, please withold your meteorological disbelief!)

In [35]:
golf_df[golf_df['Temperature'] > 70]["Windy"] = True

/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


What?

In [36]:
golf_df[golf_df['Temperature'] > 70]["Windy"]

0     False
1      True
2     False
7     False
9     False
10     True
11     True
12    False
13     True
Name: Windy, dtype: bool

**WUT?!?!**

This pattern is called chained indexing, and it is an **anti-pattern**!  Pandas can not guarentee that assignments will hold when you index twice!

To fix these issues, we need to study the other indexing options that Pandas provides.

**Note**: For a techincal overview of why this happens, the pandas documentation covers the issues with chained indexing extensively:

[Pandas Docs on Chained Indexing](https://pandas.pydata.org/pandas-docs/stable/indexing.html#returning-a-view-versus-a-copy)

### Indexers: .loc and .iloc

The professional way to do indexing in pandas is with two indexing objects:

  - `df.iloc` is **positionally based**.  This indexer accepts integers and integer slices, and essentially treats the data frame as if it was a simple matrix.
  - `df.loc` is **label based**.  This indexer works with row and column indices / labels.
  
There used to be another one, and you will encounter it sometimes

  - `df.ix` is **mixed**, it works with row numbers (integers) and column labels (names).
  
**The `ix` indexer is depreciated, and you will get a warning if you use it.  It will be removed in a future version of pandas.  Don't write code that uses ix!**

In [37]:
df = pd.DataFrame({
    'some_integers': [0, 0, 1, 1, 2, 2],
    'some_strings': ['x', 'y', 'z', 'x', 'y', 'z'],
    'some_booleans': [0, 0, 1, 0, 1, 1]},
    index=['a', 'b', 'c', 'd', 'e', 'f']
)

In [38]:
df

,some_integers,some_strings,some_booleans
a,0,x,0
b,0,y,0
c,1,z,1
d,1,x,0
e,2,y,1
f,2,z,1


In [39]:
df.iloc[2:4, 0:2]

,some_integers,some_strings
c,1,z
d,1,x


In [40]:
df.loc['b':'e', ['some_integers', 'some_booleans']]

,some_integers,some_booleans
b,0,0
c,1,1
d,1,0
e,2,1


**Deprecation Warning!!!!**

In [41]:
df.ix[2:4, ['some_integers', 'some_booleans']]

/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,some_integers,some_booleans
c,1,1
d,1,0


### Mixed Indexing

So what do we do if we want to get the rows by position, and get the columns by label?  I.e. if we have a use for **mixed indexing**.

In [42]:
# Mixed indexing with iloc: will not work.
df.iloc[2:4, ['some_integers', 'some_booleans']]

TypeError: cannot perform reduce with flexible type

Doing mixed indexing in modern pandas is a more explicit, less magic.  You need to use the `df.index` and `df.columns` attributes to explicitly turn positions into labels.

In [43]:
df = pd.DataFrame({
    'some_integers': [0, 0, 1, 1, 2, 2],
    'some_strings': ['x', 'y', 'z', 'x', 'y', 'z'],
    'some_booleans': [0, 0, 1, 0, 1, 1]},
    index=['a', 'b', 'c', 'd', 'e', 'f']
)

#### Rows by position, Columns by name

In [44]:
df

,some_integers,some_strings,some_booleans
a,0,x,0
b,0,y,0
c,1,z,1
d,1,x,0
e,2,y,1
f,2,z,1


In [45]:
df.index

Index(['a', 'b', 'c', 'd', 'e', 'f'], dtype='object')

In [46]:
df.index[2:4]

Index(['c', 'd'], dtype='object')

In [47]:
df.loc[df.index[2:4], ['some_integers', 'some_booleans']]

,some_integers,some_booleans
c,1,1
d,1,0


#### Rows by name, Columns by position

In [48]:
df.columns[[0, 2]]

Index(['some_integers', 'some_booleans'], dtype='object')

In [49]:
df.loc[['c', 'd'], df.columns[[0, 2]]]

,some_integers,some_booleans
c,1,1
d,1,0


### Transforming data

Arithmetic operations apply to `Series` element by element.

In [50]:
# Yes, this makes no sense.
golf_df["TempHumid"] = golf_df['Temperature'] + golf_df['Humidity']

In [51]:
golf_df.head()

,Date,Outlook,Temperature,Humidity,Windy,Result,TempHumid
0,07-01-2014,sunny,85,85,False,Don't Play,170
1,07-02-2014,sunny,80,90,True,Don't Play,170
2,07-03-2014,overcast,83,78,False,Play,161
3,07-04-2014,rain,70,96,False,Play,166
4,07-05-2014,rain,68,80,False,Play,148


In [52]:
# More Usefully

# Heat index formula taken from wikipedia: 
#    https://en.wikipedia.org/wiki/Heat_index
temp = golf_df['Temperature']
humid = golf_df['Humidity']
golf_df['HeatIndex'] = (-42.37 + 2.05*temp + 10.14*humid
                        - 0.225 * temp*humid
                        - 6.84e-3 * temp**2 
                        - 5.482e-2 * humid**2
                        + 1.23e-3 * temp**2 * humid
                        + 8.53e-4 * temp * humid**2
                        - 1.99e-6 * temp**2 * humid**2
)
golf_df[['Temperature', 'Humidity', 'HeatIndex']]

,Temperature,Humidity,HeatIndex
0,85,85,98.004631
1,80,90,84.474400
2,83,78,89.669911
3,70,96,62.847024
4,68,80,69.089776
5,65,70,73.668025
6,64,65,75.987116
7,72,95,66.247396
8,69,70,72.843649
9,75,80,74.557000


We can create a new Series by applying functions to an existing Series

In [53]:
# Create an indicator variable out of a column.
golf_df['Result'].apply(lambda x: 1 if x == 'Play' else 0)

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Result, dtype: int64

Though the previous result is better executed as

In [73]:
golf_df['Result'] == 'Play'

0     False
1     False
2      True
3      True
4      True
5     False
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13    False
Name: Result, dtype: bool

If we want an integer result, we can use `astype`.

In [74]:
(golf_df['Result'] == 'Play').astype(int)

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Result, dtype: int64

We can check that these give the same things

In [75]:
apply_result = golf_df['Result'].apply(lambda x: 1 if x == 'Play' else 0) 
broadcast_result = (golf_df['Result'] == 'Play').astype(int)

apply_result == broadcast_result

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Result, dtype: bool

### Aggregating data

The way pandas handles grouping and aggregating data is an interesting example of object oriented programming.

In [58]:
golf_df

,Date,Outlook,Temperature,Humidity,Windy,Result,TempHumid,HeatIndex
0,07-01-2014,sunny,85,85,False,Don't Play,170,98.004631
1,07-02-2014,sunny,80,90,True,Don't Play,170,84.474400
2,07-03-2014,overcast,83,78,False,Play,161,89.669911
3,07-04-2014,rain,70,96,False,Play,166,62.847024
4,07-05-2014,rain,68,80,False,Play,148,69.089776
5,07-06-2014,rain,65,70,True,Don't Play,135,73.668025
6,07-07-2014,overcast,64,65,True,Play,129,75.987116
7,07-08-2014,sunny,72,95,False,Don't Play,167,66.247396
8,07-09-2014,sunny,69,70,False,Play,139,72.843649
9,07-10-2014,rain,75,80,False,Play,155,74.557000


In [59]:
groups = golf_df.groupby('Outlook')

In [60]:
type(groups)

pandas.core.groupby.groupby.DataFrameGroupBy

A `DataFrameGroupBy` object is a weird thing.  Let's see what we can do with it.

First, the object is iterable, i.e. we can use it in a for loop.  When we do, we get pairs consisting of the label of a group, and the data frame we would get by subseting to that group.

In [76]:
for group_id, group in groups:
    print('Group Name: ', group_id)
    print('Group Data:\n', group)
    print('\n')

Group Name:  overcast
Group Data:
           Date   Outlook  Temperature  Humidity  Windy Result  TempHumid  \
2   07-03-2014  overcast           83        78  False   Play        161   
6   07-07-2014  overcast           64        65   True   Play        129   
11  07-12-2014  overcast           72        90   True   Play        162   
12  07-13-2014  overcast           81        75  False   Play        156   

    HeatIndex   DateTime  
2   89.669911 2014-07-03  
6   75.987116 2014-07-07  
11  68.106944 2014-07-12  
12  84.523441 2014-07-13  


Group Name:  rain
Group Data:
           Date Outlook  Temperature  Humidity  Windy      Result  TempHumid  \
3   07-04-2014    rain           70        96  False        Play        166   
4   07-05-2014    rain           68        80  False        Play        148   
5   07-06-2014    rain           65        70   True  Don't Play        135   
9   07-10-2014    rain           75        80  False        Play        155   
13  07-14-2014    rai

We can then apply some sort of aggregation to each subset of the data.

In [62]:
golf_df.groupby('Outlook').count()

,Date,Temperature,Humidity,Windy,Result,TempHumid,HeatIndex
Outlook,,,,,,,
overcast,4,4,4,4,4,4,4
rain,5,5,5,5,5,5,5
sunny,5,5,5,5,5,5,5


In [63]:
groups.sum()

,Temperature,Humidity,Windy,TempHumid,HeatIndex
Outlook,,,,,
overcast,300,308,2.0,608,318.287412
rain,349,406,2.0,755,350.648809
sunny,381,410,2.0,791,397.347701


In [64]:
groups.mean()

,Temperature,Humidity,Windy,TempHumid,HeatIndex
Outlook,,,,,
overcast,75.0,77.0,0.5,152.0,79.571853
rain,69.8,81.2,0.4,151.0,70.129762
sunny,76.2,82.0,0.4,158.2,79.469540


You can apply your own custom aggregation functions with `aggregate`.

In [65]:
# Get the minimum Temperature within each group.
# Note: This is an awful way to accomplish this, it's just for illustration.
def minimum_temperature(df):
    return sorted(df['Temperature'])[0]

# groups.aggregate(lambda df: sorted(df['Temperature'])[0])
groups.aggregate(minimum_temperature)

,Date,Temperature,Humidity,Windy,Result,TempHumid,HeatIndex
Outlook,,,,,,,
overcast,64,64,64,64,64,64,64
rain,65,65,65,65,65,65,65
sunny,69,69,69,69,69,69,69
